In [3]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import math

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/juno/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import get_gene_and_cohort_list_utils
import mutationSigUtils

In [39]:
multipleSampleSignatures = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/HypermutantMultipleSamplesBranchMutationalSignatures.tsv')
multipleSampleSignatures['Nmut'] = multipleSampleSignatures['Number of Mutations']

In [40]:
multipleSampleSignatures = mutationSigUtils.merge_signature_columns(multipleSampleSignatures, mode='Stratton', drop=True, mmrAgingMerge=True, smokingMerge=False, confidence=False, mean=True, prefix='Signature.')


In [41]:
multipleSampleSignatures['dominantSignature'] = multipleSampleSignatures.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='Signature.', notEnoughMuts= True), axis=1)

In [61]:
#TODO decompose at least part of this into a function that can work for clonality too
multipleSampleSignatures['pid'] = multipleSampleSignatures['Sample Name'].apply(lambda x: x[:9])
for v in set(multipleSampleSignatures['pid']):
    patients.add(v)

#Enumerates how the trunk signature 
listOfDicts = []
for patient in patients:
    patientSigs = multipleSampleSignatures[multipleSampleSignatures['pid'] == patient]
    patientSigsDict = dict(zip(patientSigs['Sample Name'], patientSigs['dominantSignature']))
    trunkKey = patient + '_trunk'
    if trunkKey in patientSigsDict:
        trunkSig = patientSigsDict[trunkKey]
        for sample, domSig in patientSigsDict.items():
            if sample != trunkKey:
                branchDomSig = patientSigs[patientSigs['Sample Name'] == sample]['dominantSignature'].iloc[0]
                if branchDomSig != 'insufficientMutBurden':
                #record how signatures change
                    trunkSigMagnitudeInTrunk = patientSigs[patientSigs['Sample Name'] == trunkKey][trunkSig].iloc[0]
                    trunkSigMagnitudeInBranch = patientSigs[patientSigs['Sample Name'] == sample][trunkSig].iloc[0]
                    branchDomSig = patientSigs[patientSigs['Sample Name'] == sample]['dominantSignature'].iloc[0]
                    branchSigMagnitudeInTrunk = patientSigs[patientSigs['Sample Name'] == trunkKey][branchDomSig].iloc[0]
                    branchSigMagnitudeInBranch = patientSigs[patientSigs['Sample Name'] == sample][branchDomSig].iloc[0]

                    transitionOccurred = 'None'
                    if trunkSig != branchDomSig:
                        transitionOccurred = trunkSig + '->' + branchDomSig
                    listOfDicts.append(
                    {'patient': patient, 'branch': sample, 'Transition': transitionOccurred, 
                    'trunkSigTMagnitude': trunkSigMagnitudeInTrunk, 'trunkSigBMagnitude': trunkSigMagnitudeInBranch,
                    'branchSigTMagnitude': branchSigMagnitudeInTrunk, 'branchSigBMagnitude': branchSigMagnitudeInBranch})

df = pd.DataFrame(listOfDicts)

In [63]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/privateVsSharedSigs.tsv', index=False, sep='\t')

In [ ]:
#
#
#Now look at differences in induction of mutations in truncal vs private

In [ ]:
#exclude P-0003650

*CLONAL/SUBCLONAL*<br><br><br><br><br><br>

In [103]:
clonalSubclonalSignatures = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/hypermutatorClonalVsSubclonalSignaturesmutationalSignatuesOutput.txt')

In [104]:
clonalSubclonalSignatures['Nmut'] = clonalSubclonalSignatures['Number of Mutations']
clonalSubclonalSignatures = mutationSigUtils.merge_signature_columns(clonalSubclonalSignatures, mode='Stratton', drop=True, mmrAgingMerge=True, smokingMerge=True, tmzMerge=True, confidence=False, mean=True, prefix='Signature.')
clonalSubclonalSignatures['dominantSignature'] = clonalSubclonalSignatures.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='Signature.', notEnoughMuts= True), axis=1)


In [105]:
clonalSubclonalSignatures['pid'] = clonalSubclonalSignatures['Sample Name'].apply(lambda x: x[:9])


In [106]:
clonalSubclonalSignatures['Tumor_Sample_Barcode'] = clonalSubclonalSignatures['Sample Name'].apply(lambda x: x[:17])

In [174]:
listOfDicts = []
for sample in set(clonalSubclonalSignatures['Tumor_Sample_Barcode']):
    sampleSigs = clonalSubclonalSignatures[clonalSubclonalSignatures['Tumor_Sample_Barcode'] == sample]
    
    if sample + '_clonal' in set(sampleSigs['Sample Name']) and sample + '_subclonal' in set(sampleSigs['Sample Name']):
        clonalDominantSig =  sampleSigs[sampleSigs['Sample Name'] == sample + '_clonal']['dominantSignature'].iloc[0]
        subclonalDominantSig = sampleSigs[sampleSigs['Sample Name'] == sample + '_subclonal']['dominantSignature'].iloc[0]
        if subclonalDominantSig != 'insufficientMutBurden':
            transitionOccurred = 'None'
            
            subclonalDomSigInClone = sampleSigs[sampleSigs['Sample Name'] == sample + '_clonal'][subclonalDominantSig].iloc[0]
            subclonalDomSigInSubclone = sampleSigs[sampleSigs['Sample Name'] == sample + '_subclonal'][subclonalDominantSig].iloc[0]
           
            if clonalDominantSig != subclonalDominantSig:
                if clonalDominantSig == 'insufficientMutBurden':
                    transitionOccurred = 'fewClonalMutations'
                else:
                    transitionOccurred = clonalDominantSig + '->' + subclonalDominantSig
            listOfDicts.append({'clonalDominantSig': clonalDominantSig, 'subclonalDominantSig': subclonalDominantSig,
                                'subclonalDomSigInClone': subclonalDomSigInClone, 'subclonalDomSigInSubclone': subclonalDomSigInSubclone,
                                'sample': sample, 'transition': transitionOccurred, 'pid': sample[:9]
                               })
    df = pd.DataFrame(listOfDicts)

In [175]:
transitionsToMark = [transition for transition, count in Counter(df['transition']).items() if count > 1]
df['transitionLabel'] = df['transition'].apply(lambda x: 
               x if x in transitionsToMark else 'other')
#df = df[df['transitionLabel'] != 'fewClonalMutations']
transitionCounter = Counter(df['transitionLabel'])
df['transitionLabel'] = df['transitionLabel'].apply(lambda x: str(x) + '_n=' + str(transitionCounter[x]))


In [176]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/subclonalVsClonalSigs.tsv', sep='\t', index=False)

In [178]:
for i in df[df['transition'] == 'fewClonalMutations']['sample']:
    print i

P-0023479-T01-IM6
P-0017697-T01-IM6
P-0017299-T01-IM6
P-0014258-T01-IM6
P-0035468-T01-IM6
P-0001229-T01-IM3
P-0026440-T01-IM6
P-0017986-T02-IM6
P-0008401-T01-IM5
P-0020069-T01-IM6
P-0006265-T01-IM5
P-0033503-T01-IM6
P-0006269-T01-IM5
P-0025735-T01-IM6
P-0017385-T01-IM6
P-0010765-T01-IM5
P-0012047-T01-IM5
P-0003182-T01-IM5
P-0004754-T01-IM5
P-0006170-T01-IM5
P-0036505-T01-IM6
P-0026678-T01-IM6
P-0016341-T01-IM6
P-0016904-T01-IM6
P-0002075-T01-IM3
P-0030708-T01-IM6
P-0020850-T01-IM6
P-0005823-T01-IM5
P-0018353-T01-IM6
P-0016247-T01-IM6
P-0011239-T02-IM5
P-0008226-T02-IM5
P-0028600-T01-IM6
P-0034095-T01-IM6
P-0002186-T01-IM3
P-0004423-T01-IM5
P-0020633-T02-IM6
P-0036924-T01-IM6
P-0009484-T01-IM5
P-0020356-T01-IM6
P-0034671-T02-IM6
P-0035180-T01-IM6
P-0013218-T01-IM5
P-0011068-T01-IM5
P-0020227-T01-IM6
P-0017841-T01-IM6
P-0004614-T01-IM5
P-0017839-T01-IM6
P-0024488-T01-IM6
P-0006753-T01-IM5
P-0021769-T01-IM6
P-0004255-T01-IM5
P-0001420-T01-IM3


In [173]:
clonalSubclonalSignatures[clonalSubclonalSignatures['Tumor_Sample_Barcode'].isin(set(['P-0002998-T01-IM3',
'P-0020042-T01-IM6']))][['Sample Name', 'Signature.MMR', 'Signature.7', 'dominantSignature']]


,Sample Name,Signature.MMR,Signature.7,dominantSignature
221,P-0002998-T01-IM3_subclonal,0.126629,0.595993,Signature.7
523,P-0020042-T01-IM6_clonal,0.364789,0.126252,Signature.AGING/MMR
525,P-0020042-T01-IM6_subclonal,0.096229,0.436326,Signature.7
820,P-0002998-T01-IM3_clonal,0.147998,0.274868,Signature.AGING/MMR


In [190]:
#
#####
###############
#####################

samDf = pd.read_table('/Users/friedman/Downloads/mut.temp.hyper.context.tsv')

In [202]:
#Counter(samDf['top_sig']).most_common(20)
#samDf.columns.values
Counter(samDf[samDf['top_sig'].isnull()]['tetanucleotideContext']).most_common(10)
#samDf[samDf['top_sig'].isnull()][['tetanucleotideContext', 'top_sig', 'Hypermutated', 'signaturesPresentInCase', 'Nmut_Mb']]

[('TCTG', 10),
 ('TCAA', 6),
 ('CCTG', 6),
 ('CCTA', 5),
 ('GCTA', 3),
 ('ACTA', 3),
 ('TCAC', 2),
 ('CTCA', 2),
 ('TCTC', 2),
 ('ACTC', 1)]